In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import json
import datetime

In [2]:
import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

### Import Data

In [3]:
matches = pd.read_csv("../Data/matches.csv")
matches.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode
0,2015-03-04,AFG,AUS,Away,3773,65
1,2012-08-25,AFG,AUS,Neutral,3437,156
2,2019-06-01,AFG,AUS,Neutral,4306,149
3,2014-03-01,AFG,BAN,Away,3623,190
4,2016-09-25,AFG,BAN,Away,3936,198


In [4]:
attributes = pd.read_csv("../Data/attributes.csv")
attributes.head()

,Full Name,Born,Bats,Bowls,Code
0,David A Warner,1986-10-27,Left Handed,Right Arm Leg Break,3637
1,Aaron James Finch,1986-11-17,Right Handed,Left Arm Orthodox,3845
2,Steven Peter Devereux Smith,1989-02-06,Right Handed,Right Arm Leg Break,3756
3,Glenn James Maxwell,1988-10-14,Right Handed,Right Arm Off Break,3969
4,James Peter Faulkner,1990-04-29,Right Handed,Left Arm Medium,3930


In [5]:
scorecard = open('../Data/scorecard.json',) 
scorecard_data = json.load(scorecard)

In [6]:
tmap = open('../Data/tmap.json',) 
tmap_data = json.load(tmap)

### Get NRR

In [7]:
def get_score(matches_data, scorecard_data):
    
    matches_data['Toss']=''
    matches_data['Score_team1']=''
    matches_data['Score_team2']=''
    
    matches_data['Result']=''
    
    rem=[]
    
    for i in range(len(matches_data)):
        
        try:
            score=scorecard_data[str(matches_data['MatchCode'].iloc[i])]['SCORES']
            
            if(score[0]>score[1]):
                matches_data['Result'].iloc[i]=1
            elif(score[1]>score[0]):
                matches_data['Result'].iloc[i]=2
            else:
                matches_data['Result'].iloc[i]=3
            
            matches_data['Score_team1'].iloc[i]=score[0]
            matches_data['Score_team2'].iloc[i]=score[1]
            
            toss = tmap_data[scorecard_data[str(matches_data['MatchCode'].iloc[i])]['TOSS']]
            
            if(toss==matches_data['Team_1'].iloc[i]):
                matches_data['Toss'].iloc[i]=1
            else:
                matches_data['Toss'].iloc[i]=2
            
        except:
            rem.append(i)
    
    #print(rem)
    matches_data.drop(rem, inplace = True)    
    return matches_data 

In [8]:
def check_allOut(scorecard_data, matchCode, team_num):
    
    bat = 'BATTING' + str(team_num)
    
    dismissal = [i[1] for i in scorecard_data[matchCode][bat]]
    
    if("not out" in dismissal or '' in dismissal):
        return False
    
    return True
    

In [9]:
def get_totalOvers(scorecard_data, matchCode, team_num):
    
    bat = 'BATTING' + str(team_num)
    
    balls = [i[3] for i in scorecard_data[matchCode][bat] if i[3]!=-1 ]
    
    overs = sum(balls)/6
    return overs

In [10]:
def get_NRR(match, scorecard_data):
    match['NRR_team1']=''
    match['NRR_team2']=''


    for i in range(len(match)):
        match_code = str(match['MatchCode'].iloc[i])
        score=scorecard_data[match_code]['SCORES']
        
        if(match['Result'].iloc[i]==0):
            match['NRR_team1'].iloc[i]=0
            match['NRR_team2'].iloc[i]=0
            
        else:
            runRate_team1=score[0]/50

            if(check_allOut(scorecard_data,match_code,2)):
                runRate_team2=score[1]/50
            else:
                if(match['Result'].iloc[i]==2):
                    runRate_team2=score[1]/get_totalOvers(scorecard_data, match_code, 2)
                else:
                    runRate_team2=score[1]/50

            #print(runRate_team1 - runRate_team2)
            match['NRR_team1'].iloc[i]= runRate_team1 - runRate_team2
            match['NRR_team2'].iloc[i]= runRate_team2 - runRate_team1
        
    return match
    

In [11]:
matches_data = get_score(matches,scorecard_data)
matches_data = get_NRR(matches_data,scorecard_data)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [12]:
matches_data.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,Toss,Score_team1,Score_team2,Result,NRR_team1,NRR_team2
0,2015-03-04,AFG,AUS,Away,3773,65,1,417,142,1,5.5,-5.5
1,2012-08-25,AFG,AUS,Neutral,3437,156,2,272,206,1,1.32,-1.32
2,2019-06-01,AFG,AUS,Neutral,4306,149,1,207,209,2,-1.80313,1.80313
3,2014-03-01,AFG,BAN,Away,3623,190,2,254,222,1,0.64,-0.64
4,2016-09-25,AFG,BAN,Away,3936,198,2,265,258,1,0.14,-0.14


### Model with NRR

In [13]:
enc = OneHotEncoder(handle_unknown='ignore')

In [14]:
enc_df = pd.DataFrame(enc.fit_transform(matches_data[['Result','Toss']]).toarray())

In [15]:
matches_data = matches_data.join(enc_df)
matches_data.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,Toss,Score_team1,Score_team2,Result,NRR_team1,NRR_team2,0,1,2,3,4
0,2015-03-04,AFG,AUS,Away,3773,65,1,417,142,1,5.5,-5.5,1.0,0.0,0.0,1.0,0.0
1,2012-08-25,AFG,AUS,Neutral,3437,156,2,272,206,1,1.32,-1.32,1.0,0.0,0.0,0.0,1.0
2,2019-06-01,AFG,AUS,Neutral,4306,149,1,207,209,2,-1.80313,1.80313,0.0,1.0,0.0,1.0,0.0
3,2014-03-01,AFG,BAN,Away,3623,190,2,254,222,1,0.64,-0.64,1.0,0.0,0.0,0.0,1.0
4,2016-09-25,AFG,BAN,Away,3936,198,2,265,258,1,0.14,-0.14,1.0,0.0,0.0,0.0,1.0


In [16]:
matches_data.rename(columns = {0: 'Win1',1: 'Win2',2: 'Tie', 3: 'Toss1',4: 'Toss2'}, inplace = True)


In [17]:
labelencoder = LabelEncoder()

matches_data['Team_1Enc'] = labelencoder.fit_transform(matches_data['Team_1'])
matches_data['Team_2Enc'] = labelencoder.fit_transform(matches_data['Team_2'])

In [18]:
venue_encoding = {
    
    "Home" : 0,
    "Away" : 1,
    "Neutral" : 2
}

In [19]:
matches_data.replace({'Venue':venue_encoding}, inplace=True)

In [20]:
matches_data.head()

,Date,Team_1,Team_2,Venue,MatchCode,GroundCode,Toss,Score_team1,Score_team2,Result,NRR_team1,NRR_team2,Win1,Win2,Tie,Toss1,Toss2,Team_1Enc,Team_2Enc
0,2015-03-04,AFG,AUS,1,3773,65,1,417,142,1,5.5,-5.5,1.0,0.0,0.0,1.0,0.0,0,0
1,2012-08-25,AFG,AUS,2,3437,156,2,272,206,1,1.32,-1.32,1.0,0.0,0.0,0.0,1.0,0,0
2,2019-06-01,AFG,AUS,2,4306,149,1,207,209,2,-1.80313,1.80313,0.0,1.0,0.0,1.0,0.0,0,0
3,2014-03-01,AFG,BAN,1,3623,190,2,254,222,1,0.64,-0.64,1.0,0.0,0.0,0.0,1.0,0,1
4,2016-09-25,AFG,BAN,1,3936,198,2,265,258,1,0.14,-0.14,1.0,0.0,0.0,0.0,1.0,0,1


In [21]:
X = matches_data[['Date','Team_1Enc','Team_2Enc','Venue','GroundCode','Toss','Toss1','Toss2']].copy()
y = matches_data[['Result','Win1','Win2','Tie','NRR_team1']]

In [22]:
# Test Train Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/5, random_state = 100)

In [23]:
X_train.head()

,Date,Team_1Enc,Team_2Enc,Venue,GroundCode,Toss,Toss1,Toss2
2347,1983-06-15,4,12,2,59,2,0.0,1.0
1398,1996-02-22,3,6,2,76,1,1.0,0.0
2100,1996-04-19,4,9,2,156,2,0.0,1.0
2563,2002-12-08,6,13,1,26,1,1.0,0.0
1520,2016-08-30,3,8,0,60,1,1.0,0.0


In [24]:
y_train.head()

,Result,Win1,Win2,Tie,NRR_team1
2347,1,1.0,0.0,0.0,1.32
1398,1,1.0,0.0,0.0,0.98
2100,1,1.0,0.0,0.0,0.76
2563,1,1.0,0.0,0.0,3.88
1520,1,1.0,0.0,0.0,3.38


In [25]:
def print_model_scores(model, data, y, predictors, target):
    '''
    A generic function to generate the performance report of the
    model in question on the data passed to it using cross-validation
    
    Args:
        model: ML Model to be checked
        data: data on which the model needs to pe trained
        predictors: independent feature variable
        target: target variable
    '''
    model.fit(data[predictors], y[target])
    predictions = model.predict(data[predictors])
    #scores = cross_val_score(model, data[predictors], y[target], scoring="neg_mean_squared_error", cv=5)
    #print('Cross-Validation Score :{}'.format(np.sqrt(-scores)))
    rms = sklearn.metrics.mean_squared_error(predictions,y[target])**0.5
    print('RMS : %s' % '{0:.2%}'.format(rms))
    #print(f"Average RMSE: {np.sqrt(-scores).mean()}")
    r2 = sklearn.metrics.r2_score(predictions,y[target])
    print('R2 : %s' % '{0:.2%}'.format(r2))
    return np.asarray(predictions)

In [26]:
def winner_prediction(model, data, y, predictors,winner):
    pred = print_model_scores(model, X_train, y_train,predictor_var, ['NRR_team1'])
    
    
    for i in range(len(pred)):
        if ((pred[i])>0):
            pred[i] = 1
        elif (pred[i]==0):
            pred[i] = 0
        else:
            pred[i] = 2

    print("Model Accuracy is: ")
    print(sum(1 for x,y in zip(pred,winner) if x == y) / len(winner))

### Model1 - Toss + GroundCode

In [27]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

<ipython-input-25-4258bb1c5d5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data[predictors], y[target])


RMS : 106.76%
R2 : -21.95%
Model Accuracy is: 
0.7482014388489209


In [28]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 161.75%
R2 : -44369.76%
Model Accuracy is: 
0.4830421377183967


In [29]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 96.68%
R2 : 44.61%
Model Accuracy is: 
0.7930798218568003


In [30]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 161.77%
R2 : -54707.61%
Model Accuracy is: 
0.4850976361767729


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


### Model2 - Toss + GroundCode + Venue

In [31]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss','Venue']
model = RandomForestRegressor(n_estimators = 100, random_state = 0)
winner_prediction(model, X_train, y_train,predictor_var, winner)

<ipython-input-25-4258bb1c5d5a>:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(data[predictors], y[target])


RMS : 106.75%
R2 : -22.20%
Model Accuracy is: 
0.7519698526892772


In [32]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss','Venue']
model = LinearRegression()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 161.75%
R2 : -44369.61%
Model Accuracy is: 
0.4830421377183967


In [33]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss','Venue']
model = DecisionTreeRegressor()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 96.68%
R2 : 44.61%
Model Accuracy is: 
0.7930798218568003


In [34]:
winner = y_train['Result']
predictor_var = ['Team_1Enc','Team_2Enc', 'GroundCode','Toss','Venue']
model = LassoCV()
winner_prediction(model, X_train, y_train,predictor_var, winner)

RMS : 161.77%
R2 : -56506.86%
Model Accuracy is: 
0.4850976361767729


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
